In [1]:
%load_ext autoreload
%autoreload 1

from dataset import Dataset, Synthetic_data

import numpy as np
from scipy.linalg import orthogonal_procrustes

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from tqdm import tqdm

from mu_dist import Mu
from sigma_dist import Sigma_Star
from gamma_dist import Gamma

from r_dist import R
from z_dist import Z
from phi_dist import Phi

from dataset_initialisation import GMM_Init

from scipy.stats import beta

In [2]:
%aimport dataset
%aimport dataset_initialisation

In [3]:
μ_1 = np.array([0.5,0.25,0.25])
μ_2 = np.array([0.4,0.15,0.45])
# μ_3 = np.array([0.3,0.45,0.25])

    # [0.8018,0.2673,0.5345] 
    # [0.2673,0.8018,0.5345]
α = 2
β = 2
prior = lambda : beta.rvs(α, β)

In [4]:
from dataset_initialisation import GMM_Init

In [5]:
# class GMM_Init():

#     def __init__(self, dataset, n_components=2):
#         self.dataset = dataset
#         gmm = GaussianMixture(n_components=n_components, covariance_type='full')
#         self.fitted_gmm = gmm.fit(dataset)

#         self.cluster_centres = self.fitted_gmm.means_
#         self.cluster_covs = self.fitted_gmm.covariances_

#         self.gamma_estimates = np.array([self.gamma_scaled(cov_mat) for cov_mat in self.cluster_covs])
    

#     def mu_prior_cov_estimate(self):
#         cluster_centres = self.fitted_gmm.means_

#         print(cluster_centres, "cluster centres")

#         return np.cov(cluster_centres.T)
    
#     def gamma_scaled(self, cov_mat):
#         ν = cov_mat[-1,-1]
#         γ = cov_mat[-1,:-1] / np.sqrt(ν)
#         return γ

    
#     def gamma_prior_cov_estimate(self):
#         print("self.gamma_estimates: ", self.gamma_estimates)
#         gamma_cov_estimate = np.cov(self.gamma_estimates.T)
#         return gamma_cov_estimate


#     def sigma_prior_params_estimate(self):
#         d = len(self.cluster_covs[0])

#         sigma_star_estimates = []

#         for i, cov_mat in enumerate(self.cluster_covs):
#             sigma_star_estimates.append(cov_mat[:-1,:-1] - np.outer(self.gamma_estimates[i], self.gamma_estimates[i]))
        
#         sigma_star_estimates = np.array(sigma_star_estimates)

#         self.sigma_star_estimates = sigma_star_estimates

#         dim = len(sigma_star_estimates[0])
#         dof = d+3 # Set it to be d+3 for now
#         sigma_star_mean = np.mean(sigma_star_estimates, axis = 0)
#         scale = sigma_star_mean * (dof - (dim + 1))

#         return scale, dof

In [6]:
ds = Synthetic_data(μ_1, μ_2, prior, N_t=1000)

In [7]:
# import matplotlib.pyplot as plt

# plt.scatter(ds.normed_embds[:,0], ds.normed_embds[:,1])

In [8]:
ds.gaussian_mm_init()

[[0.5568851  0.12722121 0.69364299]
 [0.73871968 0.46924433 0.29240905]] cluster centres
self.gamma_estimates:  [[-0.20113916  0.08445458]
 [-0.09650314  0.03301556]]
mu_cov 0 det: 7.783397529377487e-05
scale_mat 0 det: 0.008623511173730268
gamma_cov 0 det: 7.242486970835846e-06
gamma cov: [[0.00547435 0.        ]
 [0.         0.00132299]]
mu_cov 1 det: 7.783397529377487e-05
scale_mat 1 det: 0.008623511173730268
gamma_cov 1 det: 7.242486970835846e-06
gamma cov: [[0.00547435 0.        ]
 [0.         0.00132299]]


In [9]:
def print_attributes(obj):
    attributes = dir(obj)
    attributes = [attr for attr in attributes if not attr.startswith('__') and not callable(getattr(obj, attr))]
    for attr in attributes:
        value = getattr(obj, attr)
        print(f"{attr}: {value}")

## μ variables

In [10]:
print_attributes(ds.means_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.01653191 0.         0.        ]
 [0.         0.05848991 0.        ]
 [0.         0.         0.08049434]]
d: 3
k: 0
mean: [0.5568851  0.12722121 0.69364299]
prior_cov: [[0.01653191 0.         0.        ]
 [0.         0.05848991 0.        ]
 [0.         0.         0.08049434]]
prior_mean: [0. 0. 0.]

cov matrix det: 7.783397529377487e-05


In [11]:
print_attributes(ds.means_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.means_vars[0].cov))

cov: [[0.01653191 0.         0.        ]
 [0.         0.05848991 0.        ]
 [0.         0.         0.08049434]]
d: 3
k: 1
mean: [0.73871968 0.46924433 0.29240905]
prior_cov: [[0.01653191 0.         0.        ]
 [0.         0.05848991 0.        ]
 [0.         0.         0.08049434]]
prior_mean: [0. 0. 0.]

cov matrix det: 7.783397529377487e-05


## γ variables

In [12]:
print_attributes(ds.gamma_vars[0])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[0].cov))

cov: [[0.00547435 0.        ]
 [0.         0.00132299]]
d: 3
dim: 2
k: 0
mean: [-0.20113916  0.08445458]
nu: 0.03657066324587103
prior_cov: [[0.00547435 0.        ]
 [0.         0.00132299]]

cov matrix det: 7.242486970835846e-06


In [13]:
print_attributes(ds.gamma_vars[1])
print()
print("cov matrix det:" , np.linalg.det(ds.gamma_vars[1].cov))

cov: [[0.00547435 0.        ]
 [0.         0.00132299]]
d: 3
dim: 2
k: 1
mean: [-0.09650314  0.03301556]
nu: 0.04770275946491582
prior_cov: [[0.00547435 0.        ]
 [0.         0.00132299]]

cov matrix det: 7.242486970835846e-06


## sigma star variables

In [14]:
print_attributes(ds.sigma_star_vars[0])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[0].scale))

d: 3
dim: 2
dof: 6
first_moment: None
k: 0
nu: 0.03657066324587103
prior_dof: 6
prior_scale: [[ 0.08156736 -0.09517379]
 [-0.09517379  0.21677251]]
scale: [[ 0.07328338 -0.07720037]
 [-0.07720037  0.25190448]]
sigma_star_scale_det: 0.012500514391138216


In [15]:
print_attributes(ds.sigma_star_vars[1])

print("sigma_star_scale_det:", np.linalg.det(ds.sigma_star_vars[1].scale))


d: 3
dim: 2
dof: 6
first_moment: None
k: 1
nu: 0.04770275946491582
prior_dof: 6
prior_scale: [[ 0.08156736 -0.09517379]
 [-0.09517379  0.21677251]]
scale: [[ 0.08985134 -0.11314721]
 [-0.11314721  0.18164055]]
sigma_star_scale_det: 0.0035183551362845036


In [16]:
ds.gmm.print_labels()

Number of elements in each group:
Group 0: 509 elements
Group 1: 491 elements


In [18]:
ds.dataset_vi(max_iter=10)

[[0.76698329 0.05455222 0.4882223 ]
 [0.51528299 0.5463257  0.51098229]] cluster centres
self.gamma_estimates:  [[-0.16779501 -0.07135936]
 [-0.15905216 -0.10564879]]
mu_cov 0 det: 9.920911307103936e-07
scale_mat 0 det: 0.0053011410830036245
gamma_cov 0 det: 2.246812616249597e-08
gamma cov: [[3.82187543e-05 0.00000000e+00]
 [0.00000000e+00 5.87882221e-04]]
mu_cov 1 det: 9.920911307103936e-07
scale_mat 1 det: 0.0053011410830036245
gamma_cov 1 det: 2.246812616249597e-08
gamma cov: [[3.82187543e-05 0.00000000e+00]
 [0.00000000e+00 5.87882221e-04]]


Performing VI:   0%|          | 0/10 [00:00<?, ?it/s]

Performing VI: 100%|██████████| 10/10 [00:23<00:00,  2.35s/it]


In [23]:
ds.means_vars[1].mean

array([0.02443844, 0.01397415, 0.05581803])

In [22]:
ds.mean_vars[1].mean

AttributeError: 'Synthetic_data' object has no attribute 'mean_vars'